<h1>Entrenamiento Jupyter Notebooks</h1>

En el ejercicio presente usaremos una base de datos correspondiente a la 'rueda de la vida' de la empresa 'Coquito Amarillo'; en la cual se almacenan los datos capturades de un formulario para medir la satisfacción general de los empleados de la empresa en sus diferentes dimensiones de vida.

A partir de dichos datos realizaremos dos procesos: 
* Leer los datos para observarlos en pantalla, y presentar algunos elementos de estadística descriptiva a partir de ellos.
* Preparar algunos ejericios de analisis de datos y graficar sus resultados.

En este cuaderno vamos a realizar algunos pasos generales para comprender el uso de Jupyter Notebooks.
en esencia:
<ul>
    <l>Conectar a la base de datos local.</l>
    <l>Leer algunos datos existentes.</l>
    <l>Llenar algunas variables con esos datos.</l>
    <l>Procesar y construir algunos objetos graficos para facilitar el análisis de los datos extraidos.</l>
</ul>



In [2]:
from datetime import datetime

import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sqlalchemy import  text

# Configurar estilo de gráficos
sns.set(style="whitegrid")


Iiciamos con un pequeño mensaje de bienvenida:

In [3]:
print ("Hola, Bienvenido al entrenamiento de Jupyter Notebooks.")

Hola, Bienvenido al entrenamiento de Jupyter Notebooks.


 Aqui vamos a crear algunas funciones para conectarnos a la base de datos local y extraer información de las tablas existentes.


In [4]:
   
# Conexión a la base de datos MySQL
def obtener_conexion():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # Usuario de la base de datos
        password="",  # Contraseña de la base de datos
        database="pruebas"  # Nombre de la base de datos
    )

# Función para obtener los datos de las personas
def obtener_personas():
    conn = obtener_conexion()
    query = "SELECT * FROM personas"
    df_personas = pd.read_sql(query, conn)
    conn.close()
    return df_personas

# Función para obtener las respuestas de la encuesta
def obtener_encuestas():
    # A:conn = obtener_conexion()
    # A:query = "SELECT * FROM encuesta"
    from sqlalchemy import create_engine
    connection_string ="mysql+mysqlconnector://root:@localhost:3306/pruebas"
    engine = create_engine(connection_string, echo=True)
    with engine.connect() as connection:
      # B: result = connection.execute(text("SELECT * FROM encuestas WHERE id_persona = :idp"), dict(idp="9998080"))
      # B:  for row in result.mappings():
      # B:     print("fecha:" , row["fecha"])
            
    # Asegurarnos de que las columnas con fechas sean convertidas automáticamente a datetime
      df_encuestas =  connection.execute(text("SELECT * FROM encuestas"))
    # A: pd.read_sql(query, conn, parse_dates=['fecha'])  # Aquí le indicamos a pandas que convierta 'fecha' en datetime
    #A: conn.close()
    return df_encuestas

Una vez que hemos extraido los datos, vamos a hacer una visualización de ellos.


In [5]:
# Función para realizar análisis descriptivo básico de los datos de personas
def analisis_personas(df):
    descripcion = df.describe(include="all")
    print("Descripción de los datos de personas:")
    print(descripcion)
    
# Función para realizar análisis de las respuestas de la encuesta
def analisis_encuesta(df_encuestas):
    print("\nAnálisis de las respuestas de la encuesta:")
    # Calcular estadísticas básicas
    encuesta_promedio = df_encuestas.mean(axis=1)
    print(f"Promedio de respuestas por encuesta:\n{encuesta_promedio.head()}")
    
    # Descripción de las respuestas
    encuesta_descripcion = df_encuestas.describe()
    print(f"\nDescripción de las respuestas de la encuesta:\n{encuesta_descripcion}")

Así, podemos observar si los datos están adecuadamente almacenados.
Pasamos entonces a graficarlos:


In [6]:
# Función para graficar la satisfacción de los empleados con un gráfico radial (spider plot)
def graficar_satisfaccion(df_encuestas, id_persona):
    # Filtrar las respuestas de la persona
    respuestas = df_encuestas[df_encuestas['id_persona'] == id_persona].iloc[:, 2:].values.flatten()

    # Etiquetas de las dimensiones de la rueda de la vida (preguntas)
    dimensiones = ['Emocional', 'Físico', 'Relacional', 'Espiritual', 'Financiero', 'Familiar', 'Profesional', 'Social', 'Crecimiento', 'Diversión']

    # Configuración del gráfico radial
    num_vars = len(dimensiones)

    # Ángulos para las dimensiones
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()

    # Cerramos el gráfico
    respuestas = np.concatenate((respuestas, [respuestas[0]]))
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    ax.fill(angles, respuestas, color='b', alpha=0.25)
    ax.plot(angles, respuestas, color='b', linewidth=2)

    ax.set_yticklabels([])
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(dimensiones, fontsize=10, fontweight='bold')

    plt.title(f"Satisfacción de la persona {id_persona}", size=14)
    plt.show()



In [ ]:
dfp= obtener_personas()
analisis_personas(dfp)
dfe=obtener_encuestas()
analisis_encuestas(dfe)
graficar_satisfaccion(dfe,998080)
